# Recitation Course on Model Order Reduction

In [27]:
# Import packages
import numpy as np
from numpy import linalg as LA
from numpy import matrix
from numpy.linalg import inv

We consider a heat conduction problem

$$\frac{\partial T(z,t)}{\partial t} = \frac{\partial^2 T(z,t)}{\partial z^2} + h(z)u(t),\quad z\in [0,1]$$

satisfying the following boundary conditions:

$$T(0,t) = T(1,t) = 0, \quad \forall t .$$

Then the ODE equation obtained by spatial discretization is formulated as a linear
time-invariant (LTI) dynamical system of the form:

$$\frac{dx(t)}{dt} = \textbf{A}x(t) + \textbf{b}u(t),$$
$$y(t) = \textbf{c}^Tx(t),$$

where $ x = [T_1 T_2 \cdots T_N ]^T$ represents the solution at each of the interior nodes.



Our first step is to construct. Since we are dealing with the second derivative over $z$, we can use the following approximation for it 

$$\frac{\partial^2 T(z,t)}{\partial z^2} = \frac{1}{(\Delta z)^2} \left( \tilde{T}_{i+1} -2\tilde{T}_i + \tilde{T}_{i-1} \right)$$

Therefore the corresponding matrix $\textbf{A}$ is given by

$$
\textbf{A} = -\begin{bmatrix}
2 & -1 & 0 & \cdots & 0 \\
-1 & 2 & \ddots & \ddots & \vdots \\
0 & \ddots & \ddots & \ddots & 0 \\
\vdots & \ddots & \ddots & 2 & -1 \\
0 & \cdots & 0 & -1 & 2 \\
\end{bmatrix}
$$

In [73]:
N = 10
A = np.zeros((N,N))
A[0,0] = 2
A[0,1] = -1
A[N-1,N-1] = 2
A[N-1,N-2] = -1

for j in range(1,N-1):
    A[j,j] = 2
    A[j,j+1] = -1
    A[j,j-1] = -1

A = -A
print(A)

[[-2.  1. -0. -0. -0. -0. -0. -0. -0. -0.]
 [ 1. -2.  1. -0. -0. -0. -0. -0. -0. -0.]
 [-0.  1. -2.  1. -0. -0. -0. -0. -0. -0.]
 [-0. -0.  1. -2.  1. -0. -0. -0. -0. -0.]
 [-0. -0. -0.  1. -2.  1. -0. -0. -0. -0.]
 [-0. -0. -0. -0.  1. -2.  1. -0. -0. -0.]
 [-0. -0. -0. -0. -0.  1. -2.  1. -0. -0.]
 [-0. -0. -0. -0. -0. -0.  1. -2.  1. -0.]
 [-0. -0. -0. -0. -0. -0. -0.  1. -2.  1.]
 [-0. -0. -0. -0. -0. -0. -0. -0.  1. -2.]]


We have to divide with the appropriate constant

In [74]:
A = 1./N * A
print(A)

[[-0.2  0.1 -0.  -0.  -0.  -0.  -0.  -0.  -0.  -0. ]
 [ 0.1 -0.2  0.1 -0.  -0.  -0.  -0.  -0.  -0.  -0. ]
 [-0.   0.1 -0.2  0.1 -0.  -0.  -0.  -0.  -0.  -0. ]
 [-0.  -0.   0.1 -0.2  0.1 -0.  -0.  -0.  -0.  -0. ]
 [-0.  -0.  -0.   0.1 -0.2  0.1 -0.  -0.  -0.  -0. ]
 [-0.  -0.  -0.  -0.   0.1 -0.2  0.1 -0.  -0.  -0. ]
 [-0.  -0.  -0.  -0.  -0.   0.1 -0.2  0.1 -0.  -0. ]
 [-0.  -0.  -0.  -0.  -0.  -0.   0.1 -0.2  0.1 -0. ]
 [-0.  -0.  -0.  -0.  -0.  -0.  -0.   0.1 -0.2  0.1]
 [-0.  -0.  -0.  -0.  -0.  -0.  -0.  -0.   0.1 -0.2]]


Implement eigenvalue decomposition

In [75]:
lamda, v = LA.eig(A)
print(lamda)

[-0.39189859 -0.36825071 -0.33097215 -0.283083   -0.22846297 -0.17153703
 -0.00810141 -0.03174929 -0.06902785 -0.116917  ]


Our goal is to eliminate each mode, for which the ${\lambda_i}$ has a big negative real part. Remember, the transfer functions is

$$
H(s) = \sum\limits_{i=1}^N \frac{\tilde{c}_i \tilde{b}_i}{s-\lambda_i}
$$

Since we have to eliminate the "weak" modes we will sort the array of  the modes and keep the i.e. $5$ largest values.

In [76]:
Modes_sort = np.sort(lamda)[::-1]
print(Modes_sort)

Indices_sort = np.argsort(lamda)[::-1]
print(Indices_sort)

[-0.00810141 -0.03174929 -0.06902785 -0.116917   -0.17153703 -0.22846297
 -0.283083   -0.33097215 -0.36825071 -0.39189859]
[6 7 8 9 5 4 3 2 1 0]


We keep the largest $5$ elements

In [77]:
Strong_Modes = np.zeros(5)

for i in range(0,5):
    Strong_Modes[i] = lamda[Indices_sort[i]]
print(Strong_Modes)

[-0.00810141 -0.03174929 -0.06902785 -0.116917   -0.17153703]
